In [1]:
from pony.orm import *
import os
import json
import redis
import rq
import configparser
import utils

from utils import get_xxhash
import os 
import exiftool
import get_files
import xxhash
import py.path

In [2]:
db = Database(provider="mysql",
                 host="172.16.0.73",
                 user="media",
                 passwd="aterinee",
                 db="media")

class Media(db.Entity):
    xxhash = Required(str, unique=True)
    size = Required(int)
    exif = Optional(LongStr, lazy=True)
    _table_options_ = {
        'ENGINE': 'InnoDB',
        'ENCRYPTION': "'N'",
        'AUTO_INCREMENT': 10
    }
db.generate_mapping()

In [4]:
config = configparser.ConfigParser()
config.read('config.ini')

exif_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=2,
)
hash_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=3,
)

In [5]:

import_dir = "/pictures"

In [6]:
IMAGE_EXT = [".cr2", ".dng", ".jpg"]
VIDEO_EXT = [".mp4"]

In [7]:
media_files = get_files.get_files(import_dir, decorate=py.path.local)

In [14]:
for media_file in media_files:
    xxhash = get_xxhash(str(media_file))
    if exif_cache.get(xxhash) is not None:
        break

In [15]:
exif_cache.get(xxhash)

b'{"File:FileModifyDate": "2017:06:29 00:43:44-04:00", "SourceFile": "/pictures/2018-03-02T15:03:25-05:00/Android/obb/.nomedia", "ExifTool:ExifToolVersion": 10.1, "File:FilePermissions": 777, "File:Directory": "/pictures/2018-03-02T15:03:25-05:00/Android/obb", "ExifTool:Error": "Unknown file type", "File:FileInodeChangeDate": "2018:03:09 03:30:01-05:00", "File:FileAccessDate": "2018:03:09 03:41:04-05:00", "File:FileSize": 0, "File:FileName": ".nomedia"}'

In [12]:
xxhash = get_xxhash(str(media_file))
xxhash

'98451ade525cd544'

In [13]:
exif_cache.get(xxhash)

In [ ]:
a
try:
        EXIF = et.get_metadata(str(media_file))
        m = Media(
            xxhash=str(media_file),
            size=media_file.size(),
            exif=json.dumps(EXIF)
        )
    
        db.commit()
    except CacheIndexError:
        db.flush()

In [ ]:
for media_file in media_files:
    xxhash = utils.get_xxhash(str(media_file))
    if hash_cache.get(str(media_file)) is None:
        hash_cache.set(str(media_file), xxhash)
    if exif_cache.get(xxhash) is None:
        EXIF = et.get_metadata(str(media_file))
        exif_cache.set(xxhash, json.dumps(EXIF))